In [28]:
from random import randrange, random, seed
from math import exp
import glob, os
import cvrplib
import sys
from datetime import datetime

MAX_COUNT = 15000000

# Считаем суммарную Capacity для маршрута
def calc_cap(route, cd):
    sum = 0
    for i in range(1,len(route)):
        sum = sum + cd[route[i]]
    return sum

# Считаем суммарную длину маршрута
def total_length(cr,var):
    sum = 0
    for r in range(len(cr)): # r - маршрут;
        for i in range(len(cr[r])): # i - клиенты;
            if cr[r][i+1] == 0:
                break
            # Сумма евклидовых расстояний всех маршрутов
            sum = sum + var.distances[cr[r][i]][cr[r][i+1]]
    return sum

# Смотрим индекса последнего клиента на маршруте
def last_client(route):
    i = 1
    while route[i] != 0:
        i += 1
    return i

# Меняем два элемента (клиентов) местами
def cswap(cr,var,T,change):
    # Текущая длина маршрутов
    len1 = total_length( cr, var )
    # Создаём новый массив
    cr2 = [0] * len(cr)
    for i in range(len(cr)):
        cr2[i] = [0] * var.dimension
    # Поэлементно копируем весь маршрут на случай необходимости отката
    for i in range(len(cr)):
        for j in range(var.dimension):
            cr2[i][j] = cr[i][j]
    for k in range(change):
        r1 = randrange(len(cr))
        r2 = randrange(len(cr))
        # Последние колонки в маршрутах
        lc1 = last_client(cr[r1])+1
        lc2 = last_client(cr[r2])+1
        # Выбираем рандомных клиентов из маршрутов (может быть и в рамках одного маршрута)
        i1 = randrange(1,lc1)
        i2 = randrange(1,lc2)

        if( (cr[r1][i1] != 0) & (cr[r2][i2] != 0) ): # Меняем клиента на клиента, если в маршруте есть хотя бы по 1 клиенту
            if cr[r1][i1] != cr[r2][i2]:
                swap = cr[r1][i1]
                cr[r1][i1] = cr[r2][i2]
                cr[r2][i2] = swap
                #len2 = total_length( cr, var )
        else: # Перебрасываем клиента с маршрута на маршрут
            if (cr[r1][i1] + cr[r2][i2]) != 0:
                if( r1 != r2 ): # Действие имеет смысл только между разными маршрутами
                    # Ищем пустой маршрут среди двух
                    if (cr[r1][i1]) == 0: 
                        cr[r1][i1] = cr[r2][i2]
                        i = i2+1
                        while cr[r2][i] != 0: # Перемещаем клиентов влево
                            cr[r2][i-1] = cr[r2][i]
                            cr[r2][i] = 0
                            i += 1
                    else:
                        cr[r2][i2] = cr[r1][i1]
                        i = i1+1
                        while cr[r1][i] != 0: # Перемещаем клиентов влево
                            cr[r1][i-1] = cr[r1][i]
                            cr[r1][i] = 0
                            i += 1

    ret = 1
    for r in range(len(cr)):
        if calc_cap(cr[r],var.demands) > var.capacity:
            ret = 0
            break
    if ret == 1:
        len2 = total_length(cr,var)
        if (len2 <= len1):
            ret = 1
        else:
            if len2 <= T:
                ret = randrange(2)
                if ret == 1:
                    print('\n',len1,"->",len2)
            else:
                ret = 0
            #print('\r               \r',len2,end='\r')
    if ret == 0:
        for i in range(len(cr)):
            for j in range(var.dimension):
                cr[i][j] = cr2[i][j]
    return ret # 0 - неудачный обмен, 1 - удачный обмен

entries = glob.glob('datafiles\\*.vrp')
seed(0)
for file in entries:
    var = cvrplib.read(file)
    trucks = int(var.name[var.name.find('-k')+2:])
    #cap = var.capacity
    #n= var.dimension
    fr = open(file)
    skip = fr.readline()    # NAME
    val = fr.readline()     # COMMENT
    data = val.split(" ")
    #trucks = int(data[len(data)-4][:-1])
    goal = int(data[len(data)-1][:-2])
    fr.close()
    start_time = datetime.now()
    cr = [0] * trucks       # Массив с маршрутами
    print()
    print("Name:", var.name, " trucks:", trucks, " cap:", var.capacity, " var.dimension:", var.dimension, " goal:", goal, " sum:", sum(var.demands), sep='')
    for i in range(trucks):
        cr[i] = [0] * var.dimension

    # В случае неуспеха повторяем попытку с другим начальным распределением
    distr = False
    while distr == False:
        for r in range(trucks): # Обнуляем массив
            for i in range(var.dimension):
                cr[r][i] = 0
        # Случайное распределение клиентов по маршрутам с учетом вместимости
        for i in range(1,var.dimension):
            ok = False # i-тый клиент получил свой маршртут
            r = -1 # Признак того, что маршрут ещё не был присвоен
            '''
            Возможна ситуация, когда очередного клиента невозможно поместить в очередной маршрут.
            В этом случае нужно начать распределение заново.
            '''
            while ok == False:
                if r == -1:
                    r = randrange(trucks)
                    fail = 0
                else:
                    r += 1
                    r = r % trucks
                #j = 1
                #while cr[r][j] != 0: # Ищем первую свободную ячейку для очередного клиента
                #    j = j + 1
                j = last_client(cr[r])
                cr[r][j] = i
                if calc_cap(cr[r],var.demands) > var.capacity: # Проверяем, помещается ли клиент в маршрут
                    cr[r][j] = 0
                    fail += 1
                    if fail > trucks:
                        i = 999
                        break
                else:
                    ok = True
        if i != 999:
            distr = True

    fixed = 0 # Сколько шагов подряд состояние не улучшается (перестановка не совершается)
    T = total_length( cr, var ) # Начальная ""температура"
    print("T=",T)
    len2 = T
    for count in range(MAX_COUNT):
        # Меняем 2 маршрута
        len1 = len2
        ret = cswap(cr,var,(len2-goal)*(MAX_COUNT-count)/MAX_COUNT+goal,2)
        len2 = total_length( cr, var )
        if len2 <= goal:
            break;
        if len2 != len1:
            print('\r',total_length( cr, var ),end='\r')
            sys.stdout.flush()
        #print ('\n','                       \r',len2, count, fixed, end='\r')
    end_time = datetime.now()
    for r in range(trucks): # Вывод результата
        print ("Route #",r+1, sep='', end=': ') 
        for i in range(1,len(cr[r])):
            if cr[r][i] != 0:
                print(cr[r][i], end=' ')
                sys.stdout.flush()
        print()
    if len2 > goal:
        print( "FAILED: ", )
    print( "Cost", len2, count )
    print( end_time - start_time )


Name:A-n32-k5 trucks:5 cap:100 var.dimension:32 goal:784 sum:410
T= 1863
Route #1: 27 24 14 29 10 25 
Route #2: 1 13 31 19 17 
Route #3: 20 5 15 9 22 26 30 
Route #4: 12 2 3 23 8 11 4 
Route #5: 16 7 6 28 18 18 21 
Cost 778 2291
0:00:00.182512

Name:A-n33-k5 trucks:5 cap:100 var.dimension:33 goal:661 sum:446
T= 1756
Route #1: 2 64 67832 13 5 25 12 
Route #2: 22 15 28 18 11 6 
Route #3: 3 9 17 26 7 8 
Route #4: 23 19 1 14 21 31 29 24 
Route #5: 20 4 27 10 30 16 
Cost 648 5116
0:00:00.396548

Name:A-n33-k6 trucks:6 cap:100 var.dimension:33 goal:742 sum:541
T= 1527
Route #1: 1 5 26 24 23 31 25 
Route #2: 13 18 21 16 
Route #3: 28 27 30 11 29 
Route #4: 10 32 7 2 3 15 9 
Route #5: 14 19 20 8 4 22 
Route #6: 6 17 12 
Cost 741 2016
0:00:00.181549

Name:A-n34-k5 trucks:5 cap:100 var.dimension:34 goal:778 sum:460
T= 1580
Route #1: 21 32 13 10 12 3 
Route #2: 28 25 31 19 17 7 29 
Route #3: 14 15 8 6 1 11 
Route #4: 9 22 16 20 26 4 
Route #5: 27 23 24 30 5 18 2 33 
Cost 771 5961
0:00:00.443078


Route #8: 44 15 3 6 51 66 57 25 47 4 
Route #9: 40 39 8 38 68 13 
Cost 1138 348750
0:00:57.433921

Name:A-n80-k10 trucks:10 cap:100 var.dimension:80 goal:1763 sum:942
T= 3966
Route #1: 21 28 79 18 14 10 
Route #2: 63 34 2 37 48 52 46 54 
Route #3: 40 62 24 6 64 55 26 35 
Route #4: 1 59 17 31 25 41 15 33 
Route #5: 71 23 44 3 51 73 58 76 
Route #6: 38 70 53 77 29 74 5 27 78 8 
Route #7: 13 7 11 61 68 16 43 75 
Route #8: 30 57 19 65 69 9 56 47 20 
Route #9: 42 67 72 4 50 39 60 12 
Route #10: 49 36 66 22 45 32 
Cost 1759 17357
0:00:03.575851

Name:B-n31-k5 trucks:5 cap:100 var.dimension:31 goal:672 sum:412
T= 1053
Route #1: 21 29 8 7 22 24 
Route #2: 2 15 13 9 25 16 
Route #3: 23 27 10 20 
Route #4: 5 30 4 28 26 17 
Route #5: 18 3 19 11 6 12 14 1 
Cost 668 283
0:00:00.043442

Name:B-n34-k5 trucks:5 cap:100 var.dimension:34 goal:788 sum:457
T= 1479
Route #1: 1 19 9 16 14 
Route #2: 2 21 7 29 24 27 
Route #3: 5 20 33 32 22 
Route #4: 13 28 3 10 23 12 15 31 17 25 
Route #5: 8 4 26 18 6 30 11

Route #1: 90 19 151732 20 51 3 77 76 54 55 25 
Route #2: 69 1 68 80 72 21 40 60 
Route #3: 12 33 81 79 29 24 52 18 
Route #4: 73 23 74 75 22 41 15 42 
Route #5: 78 34 35 66 30 10 63 
Route #6: 94 96 99 61 16 84 5 
Route #7: 6 87 4 56 39 67 
Route #8: 83 93 85 100 14 
Route #9: 88 62 7 8 45 17 86 
Route #10: 89 82 48 47 46 36 
Route #11: 58 95 92 97 59 91 44 43 
Route #12: 53 26 13 2 57 37 98 38 
Route #13: 70 31 64 49 19 11 
Route #14: 27 28 50 9 71 65 
Cost 1069 129546
0:00:39.605099

Name:E-n101-k8 trucks:8 cap:200 var.dimension:101 goal:817 sum:1458
T= 3213
Route #1: 89 95 1682 1644 118218 19 48 46 45 17 86 14 42 43 15 
Route #2: 53 4 25 39 67 23 56 75 73 73 73 73 
Route #3: 70 30 51 9 33 79 77 3 68 80 76 50 1 69 
Route #4: 94 99 93 92 97 87 2 57 41 22 74 72 21 
Route #5: 27 28 26 12 54 55 24 29 34 78 81 35 71 65 
Route #6: 20 66 32 11 47 36 49 64 63 90 10 62 
Route #7: 6 96 59 37 100 98 85 60 5 16 44 38 
Route #8: 52 31 88 7 82 8 83 84 61 91 95 13 58 40 
Cost 816 355123
0:01:19.469

KeyboardInterrupt: 